In [ ]:
from pystac_client import Client
from odc.stac import load

import numpy as np
import xarray as xr
import folium

In [ ]:
# Fiji bounding box
lower_left =  -18.125, 177.275
upper_right = -18.075, 177.325
bbox = [lower_left[1], lower_left[0], upper_right[1], upper_right[0]]

# Create a STAC client
client = Client.open("https://earth-search.aws.element84.com/v1")

items = client.search(
    collections=["sentinel-2-c1-l2a"],
    bbox=bbox,
    datetime="2024-07/2024-09",
    query={"eo:cloud_cover": {"lt": 30}},
).item_collection()

print(f"Found {len(items)} items")

In [ ]:
data = load(
    items,
    chunks={},
    bbox=bbox,
    groupby="solar_day",
).compute()

# nodata, cloud shadow, medium cloud, high cloud
mask_flags = [1, 3, 8, 9]
cloud_mask = ~data.scl.isin(mask_flags)
masked = data.where(cloud_mask).drop_vars("scl")

scaled = (masked.where(masked != 0) * 0.0001).clip(0, 1)

scaled

In [ ]:
scaled[["red", "green", "blue"]].to_array().plot.imshow(col="time", col_wrap=2, robust=True, size=5)

In [ ]:
# water body id
def wbi(r, g, b, nr, s1, s2, filter_uabs=True):
    ws = np.zeros(r.shape)

    ndvi = (nr - r) / (nr + r)
    mndwi = (g - s1) / (g + s1)
    ndwi = (g - nr) / (g + nr)
    ndwi_l = (nr - s1) / (nr + s1)
    aweish = b + 2.5 * g - 1.5 * (nr + s1) - 0.25 * s2
    aweinsh = 4 * (g - s1) - (0.25 * nr + 2.75 * s1)
    dbsi = ((s1 - g) / (s1 + g)) - ndvi

    ws_one_logic = [
        mndwi > 0.2,
        ndwi > 0.2,
        aweinsh > 0.1879,
        aweish > 0.1112,
        ndvi < -0.2,
        ndwi_l > 1,
    ]

    ws_one = np.logical_or.reduce(ws_one_logic)
    ws = np.where(ws_one, 1, ws)

    ws_and_logic = [
        np.logical_and.reduce([filter_uabs and ws == 1]),
        np.logical_or.reduce([aweinsh <= -0.03, dbsi > 0]),
    ]

    ws_zero = np.logical_and.reduce(ws_and_logic)
    ws = np.where(ws_zero, 0, ws)

    return xr.DataArray(ws, coords=r.coords, dims=r.dims)


# pSDB calc, denum:green or red
def getPsdb(b, denum, n):
    return (np.log(n * b)) / (np.log(n * denum))


# SDB calc
def getSdb(pSDB, m0, m1):
    return m1 * pSDB - m0


# eval
def get_sdb(data, m0, m1, use_green=True, nConst=1000, mask_by_water=True):
    nri = data.nir
    s1_i = data.swir16
    s2_i = data.swir22
    bi = data.blue
    gi = data.green
    ri = data.red

    deNumi = ri if not use_green else gi

    # Water
    w = wbi(ri, gi, bi, nri, s1_i, s2_i)

    # Probability of SDB?
    psdb = getPsdb(bi, deNumi, nConst)

    # SDB values
    sdb = getSdb(psdb, m0, m1)

    results = psdb.to_dataset(name="psdb")
    results["sdb"] = sdb * -1

    if mask_by_water:
        results["sdb"] = results.sdb.where(w.mean("time") > 0.1)

    return results

In [ ]:
m1 = 155.86
m0 = 146.46

results = get_sdb(scaled, m0, m1, use_green=True, nConst=1000, mask_by_water=False)

In [ ]:
results.sdb.plot.imshow(col="time", col_wrap=2, robust=True, size=5)

In [ ]:
final = results.sdb.mean("time").to_dataset(name="sdb_mean")
final["sdb_std"] = results.sdb.std("time")
for band in ["red", "green", "blue"]:
    final[band] = scaled[band].median("time")

In [ ]:
m = folium.Map(location=[-18.1, 177.3], zoom_start=14, layer_control=True)

visual = final.odc.to_rgba(["red", "green", "blue"], vmin=0, vmax=0.3)
visual.odc.add_to(m, name="RGB")


final.sdb_std.odc.add_to(m, name="SDB Std Dev", cmap="Reds")
final.sdb_mean.odc.add_to(m, name="SDB Mean", cmap="Blues")

# Layer control
folium.LayerControl().add_to(m)

m

In [ ]:
final.sdb_mean.odc.write_cog("sdb_mean.tif", overwrite=True)